## Homework 08 - Deep Learning

### Import

In [18]:
# import libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Tensorflow libraries
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [19]:
# setting seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [20]:
# Create an instance of ImageDataGenerator with a preprocessing function
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [21]:
# Create a data generator for traning images
train_ds = train_gen.flow_from_directory(
    # Path to the directory with the training images
    'data/raw/data/train', 
    # Resize all images to 150x150
    target_size=(200,200), 
    # Number of images to be yielded in each batch
    batch_size=20,
    class_mode='categorical',
    shuffle = True,
    )

Found 800 images belonging to 2 classes.


### Loading the images


In [22]:
# Create an instance of ImageDataGenerator with a preprocessing function
train_gen = ImageDataGenerator(rescale=1./255)

In [23]:
# Get the next batch of images and labels from the generator
X, y = next(train_ds)

In [24]:
# create a data generator for validation images with prepocessing
val_gen = ImageDataGenerator(rescale=1./255)

# Create a data generator for validation images
val_ds = val_gen.flow_from_directory(
    'data/raw/data/test', 
    target_size=(200,200), 
    batch_size=20,
    class_mode='categorical',
    shuffle=True
    )

Found 201 images belonging to 2 classes.


In [30]:
from tensorflow.keras import layers, models, optimizers

# Function to define model by adding new dense layer and dropout
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                          include_top=False,
                          input_shape=(200,200,3))

    base_model.trainable = False
    
    #########################################
    
    inputs = tf.keras.Input(shape=(200,200,3))
    base = base_model(inputs, training=False)
    vectors = tf.keras.layers.GlobalAveragePooling2D()(base)
    inner = tf.keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = tf.keras.layers.Dropout(droprate)(inner) # add dropout layer
    outputs = tf.keras.layers.Dense(2)(drop)  # Change the number of output units to 2
    model = tf.keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

    def make_model(learning_rate=0.002, momentum=0.8):
        model = models.Sequential()
        model.add(layers.Input(shape=(200, 200, 3)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        optimizer = optimizers.SGD(lr=learning_rate, momentum=momentum)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [31]:
model = make_model()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │       204,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,066,582 (80.36 MB)

 Trainable params: 205,102 (801.18 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [32]:
model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 664ms/step - accuracy: 0.7496 - loss: 0.7936 - val_accuracy: 0.8905 - val_loss: 0.2793
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 631ms/step - accuracy: 0.9147 - loss: 0.2332 - val_accuracy: 0.9453 - val_loss: 0.1449
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 668ms/step - accuracy: 0.9260 - loss: 0.1668 - val_accuracy: 0.9403 - val_loss: 0.1438
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 639ms/step - accuracy: 0.9541 - loss: 0.1097 - val_accuracy: 0.9403 - val_loss: 0.1246
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 635ms/step - accuracy: 0.9454 - loss: 0.1321 - val_accuracy: 0.9154 - val_loss: 0.2229
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 627ms/step - accuracy: 0.9687 - loss: 0.0988 - val_accuracy: 0.9602 - val_loss: 0.0971
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 621ms/step - accuracy: 0.9741 - loss: 0.0566 - val_accuracy: 0.9552 - val_loss: 0.1167
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 629ms/step - accuracy: 0.9668 - loss: 0.0773 - val_accu

In [33]:
# Train the model and store the history
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

# Calculate the median of training accuracy
training_accuracies = history.history['accuracy']
median_training_accuracy = np.median(training_accuracies)
print(f"Median Training Accuracy: {median_training_accuracy}")

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 26s 640ms/step - accuracy: 0.9572 - loss: 0.1256 - val_accuracy: 0.9652 - val_loss: 0.1239
Epoch 2/10
11/40 ━━━━━━━━━━━━━━━━━━━━ 14s 509ms/step - accuracy: 0.9657 - loss: 0.1232

KeyboardInterrupt: 

In [ ]:
# Create an instance of ImageDataGenerator with the specified augmentations
train_gen_augmented = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a data generator for training images with augmentations
train_ds_augmented = train_gen_augmented.flow_from_directory(
    'data/raw/data/train', 
    target_size=(200,200), 
    batch_size=20,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
from tensorflow.keras import layers, models, optimizers

# Function to define model by adding new dense layer and dropout
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                          include_top=False,
                          input_shape=(200,200,3))

    base_model.trainable = False
    
    #########################################
    
    inputs = tf.keras.Input(shape=(200,200,3))
    base = base_model(inputs, training=False)
    vectors = tf.keras.layers.GlobalAveragePooling2D()(base)
    inner = tf.keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = tf.keras.layers.Dropout(droprate)(inner) # add dropout layer
    outputs = tf.keras.layers.Dense(2)(drop)  # Change the number of output units to 2
    model = tf.keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

    def make_model(learning_rate=0.002, momentum=0.8):
        model = models.Sequential()
        model.add(layers.Input(shape=(200, 200, 3)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        optimizer = optimizers.SGD(lr=learning_rate, momentum=momentum)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [ ]:
# Train the model and store the history
history = model.fit(
    train_gen_augmented,
    epochs=10,
    validation_data=train_ds_augmented
)

# Calculate the median of training accuracy
training_accuracies = history.history['accuracy']
average_training_accuracy = np.average(training_accuracies)
print(f"Median Training Accuracy: {average_training_accuracy}")